Created on Mon Jan 13 16:00:47 2020
<br>
Group 7
<br>
@authors : G.H.

In [16]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from shutil import copyfile
import shutil

# Deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
import pickle

In [ ]:
%run g7_functions_for_models.ipynb

In [24]:
seatguru_path = './../Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
new_path_train = './../G7_SEATGURU/Int/data_train'
new_path_val = './../G7_SEATGURU/Int/data_test'
new_paths = [new_path_train, new_path_val]

# Images parameters
size = (224, 224)
greys = False

# Data preparation

In [25]:
# Reading of csv file SEATGURU annontate
df_seat_annot = pd.read_csv('./../CSV_annotate/SEATGURU/g7_SEATGURU_annotate.csv', sep=';')
df_seat_annot

,name,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type,view
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,jpg,540,960,0.5625,3,Boeing,777,Meal
1,KLM_Airbus_A330-300_1.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int
2,American_Airlines_Boeing_767-300_3.jpg,jpg,720,720,1.0000,3,Boeing,767,Int
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,jpg,720,960,0.7500,3,Boeing,767,Meal
4,United_Airlines_Q400_A_2.jpg,jpg,720,960,0.7500,3,Other,NaN,Ext
...,...,...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,jpg,720,960,0.7500,3,Airbus,A330,Int
2552,Qatar_Airways_Airbus_A321_2.jpg,jpg,720,960,0.7500,3,Airbus,A321,Int
2553,Frontier_Airlines_Airbus_A320_4.jpg,jpg,720,720,1.0000,3,Airbus,A320,Ext_Int
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,jpg,720,960,0.7500,3,Airbus,A320,Int


In [26]:
# Get Airbus and Boeing images names for Interior view
df_airbus = df_seat_annot[df_seat_annot['aircraft_manufacturer']  == 'Airbus']
list_airbus = df_airbus[df_airbus['view'] == 'Int']['name'].tolist()

df_boeing = df_seat_annot[df_seat_annot['aircraft_manufacturer']  == 'Boeing']
list_boeing = df_boeing[df_boeing['view'] == 'Int']['name'].tolist()

In [28]:
# Create train and test sets
split_train_test_seatguru_man(new_paths, seatguru_path, list_airbus, list_boeing)

./../G7_SEATGURU/Int/data_train/Airbus
./../G7_SEATGURU/Int/data_train/Boeing


In [29]:
# Image data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(new_paths[0],
                                              target_size=size,
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)

test_generator = datagen.flow_from_directory(new_paths[1],
                                             target_size=size,
                                             color_mode='rgb',
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=True)

Found 1123 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


# Transfer learning

In [30]:
# Get the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(size[0], size[1], 1 if greys else 3))
x = base_model.output

# Add layers
x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)  # vector

# Fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
# Dense has the same number of neurons as the number of classes to predict
predictions = Dense(nb_types, activation = 'softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done after setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [31]:
# AveragePooling
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=1,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/1
24/35 [===================>..........] - ETA: 1:58 - loss: 0.9529 - accuracy: 0.4899

KeyboardInterrupt: 

In [117]:
# MaxPooling
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/10
35/35 [==============================] - 225s 6s/step - loss: 0.8825 - accuracy: 0.5335 - val_loss: 2.7981 - val_accuracy: 0.5415
Epoch 2/10
35/35 [==============================] - 228s 7s/step - loss: 0.3948 - accuracy: 0.8323 - val_loss: 5.1430 - val_accuracy: 0.5270
Epoch 3/10
35/35 [==============================] - 242s 7s/step - loss: 0.1572 - accuracy: 0.9404 - val_loss: 0.7036 - val_accuracy: 0.5747
Epoch 4/10
35/35 [==============================] - 237s 7s/step - loss: 0.1389 - accuracy: 0.9496 - val_loss: 4.6922 - val_accuracy: 0.5498
Epoch 5/10
35/35 [==============================] - 232s 7s/step - loss: 0.2218 - accuracy: 0.9184 - val_loss: 1.3458 - val_accuracy: 0.5394

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/10
35/35 [==============================] - 233s 7s/step - loss: 0.1241 - accuracy: 0.9578 - val_loss: 0.7137 - val_accuracy: 0.5353
Epoch 7/10
35/35 [==============================] - 233s 7s/step - loss: 

In [ ]:
path_mod = './../Models/'
save_model_classes(path_mod, 'Int_man', train_generator, model)